## Описание проекта

Вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

### Шаги для выбора локации обычно такие:

 - В избранном регионе собирают характеристики для скважин: качество нефти и объём её запасов;
 - Строят модель для предсказания объёма запасов в новых скважинах;
 - Выбирают скважины с самыми высокими оценками значений;
 - Определяют регион с максимальной суммарной прибылью отобранных скважин.

Предоставлены пробы нефти в трёх регионах. Характеристики для каждой скважины в регионе уже известны.

### Описание данных

Данные геологоразведки трёх регионов находятся в файлах:

id — уникальный идентификатор скважины;
f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
product — объём запасов в скважине (тыс. баррелей).

### Задача

 - Построить модель для определения региона, где добыча принесёт наибольшую прибыль.
 - Проанализировать возможную прибыль и риски техникой Bootstrap.

### Условия

Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
Бюджет на разработку скважин в регионе — 10 млрд рублей.
При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

## Шаг 1. Загрузим и исследуем данные

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import mean_squared_error as mse

revenue_per_unit = 450000 # доход за единицу продукта

In [10]:
data_region_1 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')
data_region_2 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv')
data_region_3 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv')
data_region_1.name = 'data_region_1'
data_region_2.name = 'data_region_2'
data_region_3.name = 'data_region_3'

data_all_regions = pd.concat([data_region_1, data_region_2, data_region_3])
data_list = [data_region_1, data_region_2, data_region_3]

In [17]:
for data in data_list:
    print(f"{'='*30} {data.name} {'='*30}")
    data.info()

============================== data_region_1 ==============================
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
============================== data_region_2 ==============================
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
me

Отлично, в данных отсутствуют пропуски.
Признаки и целевой столбец имеют тип данных float64.

In [13]:
for data in data_list:
    print(f"{'='*30} {data.name} {'='*30}")
    print(data.describe())

============================== data_region_1 ==============================
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347
============================== data_region_2 ==============================
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      

In [15]:
print(f"Количество скважин: {len(data_all_regions)}")
print(f"Количество уникальных скважин: {len(data_all_regions['id'].unique())}")

Количество скважин: 300000
Количество уникальных скважин: 299951


49 скважин принадлежат не одному региону. Хмм... Ну, может быть это и нормально, мало ли как они там зарегистрированы. Оставим как есть.

In [22]:
for data in data_list:
    print(f"Датасет {data.name} содержит {data.duplicated().sum()} явных дубликатов")

Датасет data_region_1 содержит 0 явных дубликатов
Датасет data_region_2 содержит 0 явных дубликатов
Датасет data_region_3 содержит 0 явных дубликатов


Явные дубли в данных отсутствуют.

## Шаг 2. Обучим и проверим модель для каждого региона

In [ ]:
for data in data_list:

In [ ]:
model = LinearRegression()